In [16]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers # helper functions

# Tensorflow libraries
import tensorflow as tf
from tensorflow import keras

In [17]:
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model, preprocessing

In [18]:
names = ['id','thick','size','shape','marg','cell_size','bare',
         'chrom','normal','mit','class']
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/' +
                 'breast-cancer-wisconsin/breast-cancer-wisconsin.data',
                names=names,na_values='?',header=None)

df.drop(columns=['id'],inplace=True)

df = df.dropna()
df.head(6)

# 2 is benign, 4 is malignant

,thick,size,shape,marg,cell_size,bare,chrom,normal,mit,class
0,5,1,1,1,2,1.0,3,1,1,2
1,5,4,4,5,7,10.0,3,2,1,2
2,3,1,1,1,2,2.0,3,1,1,2
3,6,8,8,1,3,4.0,3,7,1,2
4,4,1,1,3,2,1.0,3,1,1,2
5,8,10,10,8,7,10.0,9,7,1,4


In [19]:
df['class']=df['class'].map({4:1,2:0})
df.head(5)

,thick,size,shape,marg,cell_size,bare,chrom,normal,mit,class
0,5,1,1,1,2,1.0,3,1,1,0
1,5,4,4,5,7,10.0,3,2,1,0
2,3,1,1,1,2,2.0,3,1,1,0
3,6,8,8,1,3,4.0,3,7,1,0
4,4,1,1,3,2,1.0,3,1,1,0


In [20]:
df = df.astype(int)
df.head(5)

,thick,size,shape,marg,cell_size,bare,chrom,normal,mit,class
0,5,1,1,1,2,1,3,1,1,0
1,5,4,4,5,7,10,3,2,1,0
2,3,1,1,1,2,2,3,1,1,0
3,6,8,8,1,3,4,3,7,1,0
4,4,1,1,3,2,1,3,1,1,0


In [21]:
d = dice_ml.Data(dataframe=df, continuous_features=[],outcome_name='class')

In [22]:
train, _ = d.split_data(d.normalize_data(d.one_hot_encoded_data))
xtr = train.loc[:, train.columns != 'class']
ytr = train.loc[:, train.columns == 'class']

print(xtr)

     thick_1  thick_2  thick_3  thick_4  thick_5  thick_6  thick_7  thick_8  \
28         0        1        0        0        0        0        0        0   
686        1        0        0        0        0        0        0        0   
614        0        1        0        0        0        0        0        0   
452        0        0        1        0        0        0        0        0   
467        0        0        0        0        0        1        0        0   
..       ...      ...      ...      ...      ...      ...      ...      ...   
421        0        0        0        0        0        0        0        0   
404        1        0        0        0        0        0        0        0   
147        1        0        0        0        0        0        0        0   
248        0        0        0        1        0        0        0        0   
639        0        0        0        0        1        0        0        0   

     thick_9  thick_10  ...  normal_10  mit_1  mit_

In [23]:
#xnames = ['id','thick','size','shape','marg','cell_size','bare',
        # 'chrom','normal','mit']
#x = np.array(df[xnames])

#y=df['class'].values

In [24]:
#splitting the input into training and testing

#from sklearn.model_selection import train_test_split

#xtr, xts, ytr, yts = train_test_split(x,y, test_size=0.2)

In [25]:
#Feature Scaling
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#xtr = sc.fit_transform(xtr)
#xts = sc.transform(xts)

In [26]:
#NEURAL NET MODEL

import os
import tensorflow as tf
import tensorflow.keras.backend as K

K.clear_session()

model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation='relu', input_shape=(xtr.shape[1],), name='hidden'),
  tf.keras.layers.Dense(1, activation='sigmoid', name='output') #sigmoid output=binary classification
])


from tensorflow.keras import optimizers

opt = optimizers.Adam(lr=0.01) 
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit( xtr, ytr, batch_size = 3, epochs = 10, shuffle = True)#, validation_data=(xts,yts))

Epoch 1/10
182/182 [==============================] - 0s 2ms/step - loss: 0.1483 - accuracy: 0.9414
Epoch 2/10
182/182 [==============================] - 0s 2ms/step - loss: 0.0536 - accuracy: 0.9762: 0s - loss: 0.0478 - accuracy: 0.
Epoch 3/10
182/182 [==============================] - 0s 2ms/step - loss: 0.0282 - accuracy: 0.9927
Epoch 4/10
182/182 [==============================] - 0s 2ms/step - loss: 0.0187 - accuracy: 0.9945
Epoch 5/10
182/182 [==============================] - 0s 3ms/step - loss: 0.0107 - accuracy: 0.9963
Epoch 6/10
182/182 [==============================] - 0s 3ms/step - loss: 0.0078 - accuracy: 0.9945
Epoch 7/10
182/182 [==============================] - 0s 2ms/step - loss: 0.0060 - accuracy: 0.9963
Epoch 8/10
182/182 [==============================] - 1s 3ms/step - loss: 0.0051 - accuracy: 0.9963
Epoch 9/10
182/182 [==============================] - 1s 3ms/step - loss: 0.0040 - accuracy: 0.9963
Epoch 10/10
182/182 [==============================] - 1s 3ms/step

In [57]:
backend = 'TF'+tf.__version__[0] # TF2
m = dice_ml.Model(model=model, backend=backend)

In [58]:
# initiate DiCE
exp = dice_ml.Dice(d, m)

In [59]:
xtr

,thick_1,thick_2,thick_3,thick_4,thick_5,thick_6,thick_7,thick_8,thick_9,thick_10,...,normal_10,mit_1,mit_2,mit_3,mit_4,mit_5,mit_6,mit_7,mit_8,mit_10
28,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
686,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
614,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
452,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
467,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
404,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
147,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
248,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [60]:
rowData = xtr.loc[ 28 , : ]
rows=list(rowData)
#print(rows)


columns=[]
for col in xtr: 
    columns.append(col)
    #print(col) 

query_instance = dict(zip(columns, rows))

print(query_instance)

{'thick_1': 0, 'thick_2': 1, 'thick_3': 0, 'thick_4': 0, 'thick_5': 0, 'thick_6': 0, 'thick_7': 0, 'thick_8': 0, 'thick_9': 0, 'thick_10': 0, 'size_1': 1, 'size_2': 0, 'size_3': 0, 'size_4': 0, 'size_5': 0, 'size_6': 0, 'size_7': 0, 'size_8': 0, 'size_9': 0, 'size_10': 0, 'shape_1': 1, 'shape_2': 0, 'shape_3': 0, 'shape_4': 0, 'shape_5': 0, 'shape_6': 0, 'shape_7': 0, 'shape_8': 0, 'shape_9': 0, 'shape_10': 0, 'marg_1': 1, 'marg_2': 0, 'marg_3': 0, 'marg_4': 0, 'marg_5': 0, 'marg_6': 0, 'marg_7': 0, 'marg_8': 0, 'marg_9': 0, 'marg_10': 0, 'cell_size_1': 0, 'cell_size_2': 1, 'cell_size_3': 0, 'cell_size_4': 0, 'cell_size_5': 0, 'cell_size_6': 0, 'cell_size_7': 0, 'cell_size_8': 0, 'cell_size_9': 0, 'cell_size_10': 0, 'bare_1': 1, 'bare_2': 0, 'bare_3': 0, 'bare_4': 0, 'bare_5': 0, 'bare_6': 0, 'bare_7': 0, 'bare_8': 0, 'bare_9': 0, 'bare_10': 0, 'chrom_1': 0, 'chrom_2': 1, 'chrom_3': 0, 'chrom_4': 0, 'chrom_5': 0, 'chrom_6': 0, 'chrom_7': 0, 'chrom_8': 0, 'chrom_9': 0, 'chrom_10': 0, 'n

In [61]:
# query instance in the form of a dictionary; keys: feature name, values: feature value


#query_instance = {'thick':9, 
#                  'size':6, 
 #                 'shape':9, 
  #                'marg':2, 
#                  'cell_size':10,
 #                 'bare': 6, 
 #                 'chrom':2, 
#                  'normal': 9,
  #                'mit':4}
#data point 98, result is 1/malignant

In [63]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=2, desired_class="opposite")

IndexError: single positional indexer is out-of-bounds

In [ ]:
widget(annmodel, df1, query_instance)